This code can be used to clean ebay scrapes, following an initial parse in Roman_Coin_web_scrape- tested to work with search results for 'completedsales' and 'findItemsAdvanced'

In [91]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
%matplotlib inline

In [92]:
#read in the relevant csv to clean here after scraping
roman_coins = pd.read_csv('./uncleaned_completed_sales_3_4', index_col=0)

In [66]:
#autopay allows you to automatically pay with paypal without first going to that url

In [93]:
#12 observations are being sold with a charity ID, meaning they give the proceeds to charity
roman_coins['charityId'].value_counts()
#these are all for the same coin type/seller it appears
roman_coins = roman_coins.drop('charityId', axis=1)

In [94]:
#Only 10 observations have a conditionId 
roman_coins['condition'].value_counts()
roman_coins = roman_coins.drop('condition', axis=1)

In [95]:
#Overwhelmingly US, but may be worth keeping for now
roman_coins['country'].value_counts()

US    1855
BG     603
GB     451
UA     419
ES     282
TR     173
RS     131
CA     129
BA     107
PL      91
CY      85
AT      80
IL      73
DE      70
AU      60
NO      56
HR      46
AL      46
NZ      34
PT      26
YU      24
HU      21
JO      21
FR      19
LV      14
LT      11
SK      10
IT      10
RO       9
NL       6
GR       5
TH       5
BH       4
SI       4
VN       4
RU       3
CN       3
AM       2
ID       2
EE       2
SE       1
VA       1
PR       1
SA       1
Name: country, dtype: int64

In [96]:
#only one value here
roman_coins['discountPriceInfo'].value_counts()
roman_coins = roman_coins.drop('discountPriceInfo', axis=1)

#I already scraped photos and this column is incomplete anyways
roman_coins = roman_coins.drop('galleryPlusPictureURL', axis=1)
roman_coins = roman_coins.drop('galleryURL', axis=1)

In [97]:
#overwhelmingly ebay US (9891 of obs.)
roman_coins['globalId'].value_counts()
roman_coins = roman_coins.drop('globalId', axis=1)

#all false
roman_coins = roman_coins.drop('isMultiVariationListing', axis=1)

In [98]:
#listingInfo has lots of useful information and I want to add these variables separately
print(roman_coins['listingInfo'][0])
listings = roman_coins['listingInfo'].map(eval).apply(pd.Series)

#merging the new columns onto the dataframe
roman_coins = pd.concat([roman_coins, listings], axis=1)

#listings column now redundant
roman_coins = roman_coins.drop('listingInfo', axis=1)

{'bestOfferEnabled': 'false', 'buyItNowAvailable': 'false', 'startTime': '2018-02-25T16:31:26.000Z', 'endTime': '2018-03-04T16:31:26.000Z', 'listingType': 'Auction', 'gift': 'false'}


In [99]:
#overwhelmingly PayPal, but I will convert this to a dummy variable for now, 0 just paypal, 1 paypal or other method
roman_coins['paymentMethod'] = [0 if x == 'PayPal' else 1 for 
                                x  in roman_coins['paymentMethod']]
print(roman_coins['paymentMethod'].value_counts())

0    4846
1     154
Name: paymentMethod, dtype: int64


In [100]:
#these variables are already converted to columns
roman_coins = roman_coins.drop('sellingStatus', axis=1)

#already have this variable elsewhere
roman_coins = roman_coins.drop('primaryCategory', axis=1)

In [101]:
#I want to swap swap the values here, to make true 0 and false 1 
print(roman_coins['returnsAccepted'].value_counts())
roman_coins['returnsNotAccepted'] = roman_coins['returnsAccepted'].astype(int) * -1 + 1

#dropping old columns
roman_coins = roman_coins.drop('returnsAccepted', axis=1)

True     3926
False    1074
Name: returnsAccepted, dtype: int64


In [102]:
#these are too varied to make a dummy or make as separate variables, dropping 
roman_coins = roman_coins.drop('secondaryCategory', axis=1)

In [103]:
#the shipping information is interesting to me and I am going to keep it, affects cost 
print(roman_coins['shippingInfo'][0])
shipping = roman_coins['shippingInfo'].map(eval).apply(pd.Series)

{'shippingServiceCost': {'_currencyId': 'USD', 'value': '0.0'}, 'shippingType': 'FlatDomesticCalculatedInternational', 'shipToLocations': 'Worldwide', 'expeditedShipping': 'false', 'oneDayShippingAvailable': 'false', 'handlingTime': '1'}


In [104]:
#There are 287 null values in shippingServiceCost- they have 'calculated' shipping type
#I am going to extract relevant values for new columns

ship_cost_dicts = []
shipping_currency = []

for val in shipping['shippingServiceCost']:
    try:
        currency = val['_currencyId']
        cost = val['value']
        ship_cost_dicts.append(cost)
        shipping_currency.append(currency)
    
    except:
        
        ship_cost_dicts.append(np.nan)
        shipping_currency.append(np.nan)


#adding these columns
shipping['shipping_cost'] = ship_cost_dicts
shipping['shipping_cost_currency'] = shipping_currency

#dropping old column
shipping = shipping.drop('shippingServiceCost', axis=1)

In [105]:
#Converting NaN to mean column value in handlingTime
shipping['handlingTime'] = shipping['handlingTime'].astype(float)
shipping['handlingTime'] = shipping['handlingTime'].fillna(
                                    shipping['handlingTime'].mean())

In [106]:
#creating dummy variables for expedited shipping availability
mapper = {'false' : 0, 'true': 1}
shipping['expeditedShipping'] = shipping['expeditedShipping'].map(mapper)

#only 22 true values, dropping
shipping = shipping.drop('oneDayShippingAvailable', axis=1)

#all ship worldwide, dropping
shipping = shipping.drop('shipToLocations', axis=1)

#all USD or NAN, dropping 
shipping = shipping.drop('shipping_cost_currency', axis=1)

In [107]:
#will have to decide what to do with shipping types later on, will likely make dummies 
print(shipping['shippingType'].value_counts())

#merging shipping back into main DF 
roman_coins = pd.concat([roman_coins, shipping], axis=1)

#dropping shipping info
roman_coins = roman_coins.drop('shippingInfo', axis=1)

Flat                                   3815
Free                                    769
Calculated                              242
FlatDomesticCalculatedInternational     160
FreePickup                                9
CalculatedDomesticFlatInternational       3
NotSpecified                              2
Name: shippingType, dtype: int64


In [108]:
#only 34 subtitles, dropping
roman_coins = roman_coins.drop('subtitle', axis=1)

#converting to dummy variable 1 is toprated, 0 not toprated
roman_coins['topRatedListing'] = roman_coins['topRatedListing'].astype(int)

#Extracting the meaningful text from the item urls
roman_coins['URLTitle'] = [val[24:-13] for val in roman_coins['viewItemURL']]

#dropping itemURL from above
roman_coins = roman_coins.drop('viewItemURL', axis=1)

#category Id is sufficient, dropping
roman_coins = roman_coins.drop('categoryName', axis=1)

#all gift false, dropping
roman_coins = roman_coins.drop('gift', axis=1)

#all values are active, dropping
roman_coins = roman_coins.drop('sellingState', axis=1)

#autoPay as int dummy
roman_coins['autoPay'] = roman_coins['autoPay'].astype(int)

#turning categoryId into object variable
roman_coins['categoryId'] = roman_coins['categoryId'].astype(object)

In [109]:
#beginning to convert  time columns
roman_coins['startTime'] = pd.to_datetime(roman_coins['startTime'])
roman_coins['endTime'] = pd.to_datetime(roman_coins['endTime'])

In [110]:
#converting 'time left' to timedeltas 
#no time left for completed sales, do not run
times = []
for val in roman_coins['timeLeft']:
    x = re.sub(r'[HM]+', ':', val)
    time = re.sub(r'[SPT]', '', x)
    new_time = pd.Timedelta(time)
    times.append(new_time)

roman_coins['timeLeft'] = times

KeyError: 'timeLeft'

In [111]:
#Converting watchcount/shipping_Cost to float
roman_coins['watchCount'] = roman_coins['watchCount'].astype(float)
roman_coins['shipping_cost'] = roman_coins['shipping_cost'].astype(float)

In [112]:
#converting column into prices from dicts
buyitnow = []
for val in roman_coins['buyItNowPrice']:
    try:
        price = val['value']
        buyitnow.append(price)
    except:
        buyitnow.append(np.nan)
        
roman_coins['buyItNowPrice'] = buyitnow
roman_coins['buyItNowPrice'] = roman_coins['buyItNowPrice'].astype(float)

#all in USD, dropping
roman_coins = roman_coins.drop('convertedBuyItNowPrice', axis=1)

In [113]:
#bestoffer means that the seller accepts a sale if the price is less than the buy it now price
mapper = {'false' : 0, 'true': 1}
roman_coins['bestOfferEnabled'] = roman_coins['bestOfferEnabled'].map(mapper)
#converting to dummy variable
roman_coins['buyItNowAvailable'] = roman_coins['buyItNowAvailable'].map(mapper)

In [90]:
#roman_coins.to_csv('./cleaned_csv_3_1')
#roman_coins.to_csv('./cleaned_completed_3_4')